In [1]:
import pandas as pd
import copy
import numpy as np
import warnings
warnings.simplefilter("ignore")

# Download The Data

In [2]:
raw_data = pd.read_csv('PUMSsubset_SanDiego-2017-2021.txt', delimiter='\t')

# Adjust The Household Income 

In [3]:
def multiply_income(row):
    if row["Year"] == 2019:
        return row["HINCP"] * 344.416/299.433
    elif row["Year"] == 2020:
        return row["HINCP"] * 344.416/303.932
    elif row['Year'] == 2021:
        return row["HINCP"] * 344.416/319.761

raw_data["adjusted_income"] = raw_data.apply(multiply_income, axis=1)

# Create Income Categories

In [4]:
def income_cat(x):
    if x == 9999999:
        return 0
    elif x < 15000:
        return 1
    elif x >= 15000 and x < 30000:
        return 2
    elif x >= 30000 and x < 45000:
        return 3
    elif x >= 45000 and x < 60000:
        return 4
    elif x >= 60000 and x < 75000:
        return 5
    elif x >= 75000 and x < 100000:
        return 6
    elif x >= 100000 and x < 125000:
        return 7
    elif x >= 125000 and x < 150000:
        return 8
    elif x >= 150000 and x < 200000:
        return 9
    elif x >= 200000 and x < 300000:
        return 10
    elif x >= 300000 and x < 500000:
        return 11
    elif x >= 500000 and x < 700000:
        return 12
    elif x >= 700000 and x < 900000:
        return 13
    else:
        return 14

In [5]:
# Temp new one 
def income_cat(x):
    if x == 9999999:
        return 0
    elif x < 15000:
        return 1
    elif x >= 15000 and x < 30000:
        return 2
    elif x >= 30000 and x < 45000:
        return 3
    elif x >= 45000 and x < 60000:
        return 4
    elif x >= 60000 and x < 75000:
        return 5
    elif x >= 75000 and x < 100000:
        return 6
    elif x >= 100000 and x < 125000:
        return 7
    elif x >= 125000 and x < 150000:
        return 8
    elif x >= 150000 and x < 200000:
        return 9
    else:
        return 10

# Create Main Function

In [6]:
def create_frequency_table(df):
    # Create the income categories
    df['IncomeCat'] = df['adjusted_income'].apply(income_cat)

    # Created a weighted frequency
    weighted_freq = df.groupby('IncomeCat').apply(lambda x: np.sum(x['WGTP']) / np.sum(df['WGTP'])).reset_index(name='frequency_percent')

    # Multiple weighted frequency to read as a percent
    weighted_freq['frequency_percent'] = weighted_freq['frequency_percent'] * 100

    return weighted_freq

# Run Main Function

In [7]:
processed_data = raw_data[((raw_data['RELSHIPP'] == 20) & (raw_data['Year'] == 2019) & (raw_data['ADJINC'] > 0) & (raw_data['TYPEHUGQ'] == 1))]

In [8]:
result = create_frequency_table(df=processed_data)
result#.to_excel('weighted_distribution_2019.xlsx')
result

,IncomeCat,frequency_percent
0,1,6.412612
1,2,7.981669
2,3,7.826430
3,4,9.258228
4,5,8.204004
5,6,12.035872
6,7,10.989980
7,8,8.441687
8,9,10.847896
9,10,18.001623


# Non Weighted Frequency

In [9]:
# pd.DataFrame(processed_data['IncomeCat'].value_counts().sort_index()).to_excel('frequency_counts_2019.xlsx')

In [10]:
pd.DataFrame(processed_data['IncomeCat'].value_counts(normalize=True)).sort_index()

,IncomeCat
1,0.060745
2,0.074650
3,0.077212
4,0.087641
5,0.079682
6,0.119019
7,0.109322
8,0.084805
9,0.112981
10,0.193944


In [18]:
pd.DataFrame(processed_data['IncomeCat'].value_counts(normalize=True)).sort_index()['IncomeCat'].reset_index(drop=True)*100

0     6.074467
1     7.465008
2     7.721160
3     8.764066
4     7.968164
5    11.901930
6    10.932211
7     8.480468
8    11.298143
9    19.394383
Name: IncomeCat, dtype: float64

In [19]:
result['non_weighted_frequency'] = pd.DataFrame(processed_data['IncomeCat'].value_counts(normalize=True)).sort_index()['IncomeCat'].reset_index(drop=True)*100

In [21]:
result.columns = ['Income Category', 'Weighted Frequency', 'Non Weighted Frequency']

In [23]:
result#.to_excel('weighted_and_unweighted.xlsx')